In [ ]:
import numpy as np
import scipy.special as sysp
import sys,os

sys.path.append('../')
from mountaineer import Model,Chi2,Mountaineer

sys.path.append('/home/aseem/python_modules/MachineLearning/mlfundas/code/')
from mlalgos import BuildNN,Sequential

sys.path.append('../../picasa/code/')
from gpr_train import GPRTrainer
from picasa import PICASA

import copy
from time import time

import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib as mpl
import matplotlib.colors as pltcol
import gc

from cobaya.run import run
from cobaya.log import LoggedError
from getdist.mcsamples import loadMCSamples
import getdist.plots as gdplt

pic = PICASA()

In [ ]:
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['legend.fontsize'] = 14 # 14
mpl.rcParams['legend.labelspacing'] = 0.25
FS = 18
FS2 = 15
FS3 = 13
FSL = 22

mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.minor.size'] = 3
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.minor.size'] = 3

#mpl.rcParams.keys()

## Testing Posterior Emulation

### Power-Sine model

In [ ]:
class PowerSine(Model):
    # n_params = 3
    # def __init__(self):
    #     Model.__init__(self,n_params=self.n_params)
    def __init__(self,n_params=3,adam=True,B1_adam=0.9,B2_adam=0.999,eps_adam=1e-8):
        Model.__init__(self,n_params=n_params,adam=adam,B1_adam=B1_adam,B2_adam=B2_adam,eps_adam=eps_adam)

    def calc_model(self,X):
        out = self.params[0,0]*np.fabs(X)**self.params[1,0]*np.sin(X + self.params[2,0])**2
        return out # X.shape = (1,n_samp)
    
    def calc_dmdtheta(self):
        # self.X will be available for the data set
        arg_sin = self.X[0] + self.params[2,0]
        sinx = np.sin(arg_sin)
        absx = np.fabs(self.X[0])
        absxa1 = absx**self.params[1,0]
        dmdtheta = np.zeros((self.n_params,self.X.shape[1])) # (n_params,n_samp)
        dmdtheta[0] = absxa1*sinx**2
        dmdtheta[1] = self.params[0,0]*dmdtheta[0]*np.log(absx + 1e-20)
        dmdtheta[2] = self.params[0,0]*absxa1*np.sin(2*arg_sin)
        
        return dmdtheta


### Gaussian mixture model

In [ ]:
class GaussMix(Model):
    # n_params = 3
    # def __init__(self):
    #     Model.__init__(self,n_params=self.n_params)
    def __init__(self,n_params=3,adam=True,B1_adam=0.9,B2_adam=0.999,eps_adam=1e-8):
        Model.__init__(self,n_params=n_params,adam=adam,B1_adam=B1_adam,B2_adam=B2_adam,eps_adam=eps_adam)
        # params are (amp,mu,sig^2) for each component
        if self.n_params % 3 != 0:
            raise ValueError('n_params must be multiple of 3 in GaussMix.')
        self.n_comp = self.n_params // 3

    def calc_model(self,X):
        out = np.zeros_like(X)
        for c in range(self.n_comp):
            amp,mu,sig2 = self.params[c*3:(c+1)*3,0]
            out += amp*np.exp(-0.5*(X-mu)**2/sig2)
        return out # X.shape = (1,n_samp)
    
    def calc_dmdtheta(self):
        # self.X will be available for the data set
        dmdtheta = np.zeros((self.n_params,self.X.shape[1])) # (n_params,n_samp)
        for c in range(self.n_comp):
            amp,mu,sig2 = self.params[c*3:(c+1)*3,0]
            exp_partarg = (self.X-mu)/sig2
            exparg = (self.X-mu)*exp_partarg
            exponential = np.exp(-0.5*exparg)
            dmdtheta[3*c] = exponential
            dmdtheta[3*c+1] = amp*exp_partarg*exponential
            dmdtheta[3*c+2] = 0.5*amp/sig2*exparg*exponential
        
        return dmdtheta


## Setup data

In [ ]:
Model_Type = 'gm' # 'ps' or 'gm'

In [ ]:
rng = np.random.RandomState(1983)

X_all = np.linspace(-2,4,30)
n_samp = X_all.size
sigma = np.linspace(0.1,0.3,n_samp)

if Model_Type == 'ps':
    sigma *= 2.0
    N_walker = 10
    max_epoch = 50
    check_after = 5
    mb_count = 3
    lrate = 0.1
    id_str = 'ps' 
    model = PowerSine()
    model.params[0] = 1
    model.params[1] = 1.5
    model.params[2] = 0    
elif Model_Type == 'gm':
    sigma *= 0.5
    n_comp = 2
    N_walker = 10 if n_comp == 1 else 20
    max_epoch = 50 if n_comp == 1 else 100
    check_after = 10 if n_comp == 1 else 20
    mb_count = 3 #if n_comp == 1 else 3
    lrate = 0.1 #if n_comp == 1 else 0.1
    id_str = 'gm{0:d}c'.format(n_comp)
    model = GaussMix(n_params=3*n_comp)
    for c in range(n_comp):
        model.params[3*c] = -1 + 3*c # amp(c)
        model.params[3*c+1] = -1 + (c+0.5)**2 # mu(c)
        model.params[3*c+2] = 4/(c+3)**2 # sig2(c)

X_all = model.rv(X_all)
n_params = model.n_params
invcov_mat = np.diag(1/sigma**2)
Y_all_true = model.forward(X_all)
Y_all = Y_all_true + sigma*rng.rand(Y_all_true.shape[0],Y_all_true.shape[1])

dof = X_all.shape[1] - n_params

id_str += '_{0:d}walkers'.format(N_walker) 
print('id_str:',id_str)
file_stem = 'walks/' + id_str

In [ ]:
Walks_Exist = True

if Model_Type == 'ps':
    param_mins = [0.5,1,-0.5]
    param_maxs = [1.5,2,0.5]
    chosen_Model = PowerSine
elif Model_Type == 'gm':
    param_mins = [-1.5,-1.5,0.1]
    param_maxs = [-0.5,0,0.8] if n_comp==1 else [-0.5,0,1.2]
    if n_comp > 1:
        param_mins = param_mins + [1,0.5,0.05]
        param_maxs = param_maxs + [3,2,0.8]
    chosen_Model = GaussMix

loss_params = {'invcov_mat':invcov_mat}

dp = {'N_walker':N_walker,'file_stem':file_stem,'model':chosen_Model,'n_params':n_params,
      'param_mins':param_mins,'param_maxs':param_maxs,
      'adam':True,'B1_adam':0.9,'B2_adam':0.999,'eps_adam':1e-8,
      'X':X_all,'Y':Y_all,'val_frac':0.2,'loss':Chi2,'walks_exist':Walks_Exist,
      'seed':None,'verbose':True,'logfile':None,
      'max_epoch':max_epoch,'mb_count':mb_count,'lrate':lrate,'loss_params':loss_params,'check_after':check_after}

mnt = Mountaineer(data_pack=dp)

In [ ]:
if not Walks_Exist:
    mnt.climb()
walks = mnt.gather()
mnt.visualize(walks)
data = mnt.load()

## Emulation

### GPR

In [ ]:
GPR_Exists = True
CV_Thresh = 0.01 if Model_Type == 'ps' else 1e-3
GPR_Dir = 'gpr/stats_'+id_str
gprt = GPRTrainer(data_file=mnt.walks_file,tmp_dir=GPR_Dir)

In [ ]:
start_time = time()
Kernel = 'rbf' if Model_Type == 'ps' else ('rbf' if n_comp == 1 else 'matern')
Max_Iter = 25 if Model_Type == 'ps' else (25 if n_comp == 1 else 50)
if GPR_Exists:
    interpolator = gprt.train_gpr(verbose=True,vary_kernel=False,kernel=Kernel,skip_train=True)
else:
    interpolator = gprt.train_gpr(cv_thresh=CV_Thresh,verbose=True,vary_kernel=True,max_iter=Max_Iter,max_iter_vary=Max_Iter//5)
gprt.time_this(start_time)

### Neural network

In [ ]:
Xp = data[1:,:]
Yp = mnt.rv(data[0])

Xp.shape,Yp.shape

In [ ]:
Train = True
FileStem = 'net/NN_'+id_str
Val_Frac = 0.25 #if Model_Type == 'ps' else 0.25
Seed = 42 #if Model_Type == 'ps' else 42

N_L = 3 if Model_Type == 'ps' else (3 if n_comp == 1 else 4)
Fac = 5 if Model_Type == 'ps' else (4 if n_comp == 1 else 6)
HLay = [Fac*n_params]+[4*Fac*n_params]
if N_L == 4:
    HLay = HLay + [(Fac//2)*n_params]
LRate = 0.05 if Model_Type == 'ps' else 0.005
Max_Epoch = 5000 #if Model_Type == 'ps' else (5000 if n_comp == 1 else 15000)
Check_After = 50 if Model_Type == 'ps' else Max_Epoch 
Reg_Fun = 'none' #if Model_Type == 'ps' else 'bn' 
params_setup = {'data_dim':Xp.shape[0],'L':N_L,'n_layer':HLay+[1],'seed':Seed,'standardize':True,'reg_fun':Reg_Fun,
                'atypes':['tanh']*(N_L-1)+['lin'],'loss_type':'square','file_stem':FileStem}
params_train = {'lrate':LRate,'max_epoch':Max_Epoch,'mb_count':10,'check_after':Check_After,'val_frac':Val_Frac}    
net = Sequential(params_setup)
if Train:
    start_time = time()
    net.train(Xp,Yp,params_train)
    net.save()
    net.time_this(start_time)
else:
    net.load()
    print ('... done')

## Testing emulators

In [ ]:
print('True loss calculation')
start_time = time()
Nsamp_ev = 3000
Xp_ev = mnt.gen_latin_hypercube(Nsamp=Nsamp_ev,dim=mnt.n_params,param_mins=mnt.param_mins,param_maxs=mnt.param_maxs)
Yp_ev = np.zeros((1,Nsamp_ev))
loss_params_ev = copy.deepcopy(loss_params)
loss_params_ev['Y_full'] = mnt.Y
for n in range(Nsamp_ev):
    # model.params[:,0] = Xp_ev[n]
    model.params[:,0] = Xp_ev[n]
    Loss_ev = mnt.loss_module(loss_params_ev)
    Yp_ev[0,n] = Loss_ev.forward(model.calc_model(mnt.X))
    mnt.status_bar(n,Nsamp_ev)
Xp_ev = Xp_ev.T
mnt.time_this(start_time)

print('NN prediction for loss')
start_time = time()
Yp_ev_pred_nn = net.predict(Xp_ev)
mnt.time_this(start_time)

resid_nn = Yp_ev_pred_nn[0]/Yp_ev[0] - 1

print('GPR prediction for loss')
start_time = time()
Yp_ev_pred_gpr = gprt.predict(Xp_ev.T,interpolator)
mnt.time_this(start_time)

resid_gpr = Yp_ev_pred_gpr/Yp_ev[0] - 1

In [ ]:
plt.xscale('log')
plt.xlabel('true loss')
plt.xlabel('predicted / true - 1')
plt.ylim(-1,1)
plt.scatter(Yp_ev[0],resid_nn,s=0.8,color='r',label='NN')
plt.scatter(Yp_ev[0],resid_gpr,s=0.8,color='b',label='GPR')
plt.legend()
plt.show()

plt.figure(figsize=(5,5))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('true loss')
plt.ylabel('predicted loss')
plt.plot(np.logspace(0.5,3.5,10),np.logspace(0.5,3.5,10),'k--',lw=1)
plt.scatter(Yp_ev[0],Yp_ev_pred_nn[0],s=0.8,color='r',label='NN')
plt.scatter(Yp_ev[0],Yp_ev_pred_gpr,s=0.8,color='b',label='GPR')
plt.legend()
plt.show()

## MCMC

In [ ]:
Like_Dir = 'likes/'
Plots_Dir = 'plots/'

Run_Cobaya = True
Run_NN = False
Run_GPR = False

Max_Samples = 1000000
Rminus1_Stop = 0.05 
Rminus1_CL_Stop = 0.05 # 0.05
Rminus1_CL_Level = 0.95 # 95

Burn_In = 0

if id_str[:2] == 'ps':
    Latex_List = ['a_{0}','a_{1}','a_{2}']
    Params_List = ['a0','a1','a2']
else:
    Latex_List = []
    Params_List = []
    for c in range(n_comp):
        Latex_List.append(['A_{'+str(c+1)+'}','\\mu_{'+str(c+1)+'}','\\sigma^{2}_{'+str(c+1)+'}'])
        Params_List.append(['A'+str(c+1),'mu'+str(c+1),'sig2'+str(c+1)])
    Latex_List = [item for sublist in Latex_List for item in sublist]
    Params_List = [item for sublist in Params_List for item in sublist]

info = {}
info['params'] = {}
info['likelihood'] = {'likelihoods.Chi2Like':
                      {'python_path':Like_Dir,
                       'X':X_all,'Y':Y_all,'invcov_mat':invcov_mat}}
if id_str[:2] == 'ps':
    info['theory'] = {'example_likelihoods.PowSineTheory':
                      {'python_path':Like_Dir,
                       'X':X_all}}
    for p in range(len(Params_List)):
        ref = 0.5*(param_mins[p] + param_maxs[p])
        info['params'][Params_List[p]] = {'ref':{'min':ref-0.001,'max':ref+0.001},
                                          'prior':{'min':param_mins[p],'max':param_maxs[p]},
                                          'proposal':0.01,'latex':Latex_List[p]}
else:
    info['theory'] = {'example_likelihoods.GaussMixTheory':
                      {'python_path':Like_Dir,
                       'X':X_all,'n_comp':n_comp}}
    for c in range(n_comp):
        ref = 0.5*(param_mins[3*c] + param_maxs[3*c])
        info['params'][Params_List[3*c]] = {'ref':{'min':ref-0.001,'max':ref+0.001},
                                              'prior':{'min':param_mins[3*c],'max':param_maxs[3*c]},
                                              'proposal':0.01,'latex':Latex_List[3*c]}
        ref = 0.5*(param_mins[3*c+1] + param_maxs[3*c+1])
        info['params'][Params_List[3*c+1]] = {'ref':{'min':ref-0.001,'max':ref+0.001},
                                                  'prior':{'min':param_mins[3*c+1],'max':param_maxs[3*c+1]},
                                                  'proposal':0.01,'latex':Latex_List[3*c+1]}
        ref = 0.5*(param_mins[3*c+2] + param_maxs[3*c+2])
        info['params'][Params_List[3*c+2]] = {'ref':{'min':ref-0.001,'max':ref+0.001},
                                                  'prior':{'min':param_mins[3*c+2],'max':param_maxs[3*c+2]},
                                                  'proposal':0.01,'latex':Latex_List[3*c+2]}

info['sampler'] = {'mcmc':
                   {'learn_proposal': True,
                    'Rminus1_single_split': 4,
                    'measure_speeds': True,
                    'max_samples': Max_Samples,
                    'max_tries': 1000,
                    'Rminus1_stop': Rminus1_Stop,
                    'Rminus1_cl_stop': Rminus1_CL_Stop,
                    'Rminus1_cl_level': Rminus1_CL_Level,
                    'burn_in': Burn_In}}
info_output = 'stats/chains/'+id_str
info['output'] = info_output
info["force"] = True    

info_nn = copy.deepcopy(info)
info_nn['likelihood'] = {'likelihoods.EmulLike':
                         {'python_path':Like_Dir}}
info_nn['theory'] = {'likelihoods.NNTheory':
                         {'python_path':Like_Dir,
                          'net':net,'keys':Params_List}}
info_nn['output'] = info_output+'_nn'

info_gpr = copy.deepcopy(info)
info_gpr['likelihood'] = {'likelihoods.EmulLike':
                             {'python_path':Like_Dir}}
info_gpr['theory'] = {'likelihoods.GPRTheory':
                         {'python_path':Like_Dir,
                          'gprt':gprt,'interpolator':interpolator,'keys':Params_List}}
info_gpr['output'] = info_output+'_gpr'


In [ ]:
if Run_Cobaya:
    start_time = time()
    updated_info, sampler = run(info)
    Neval_cobaya = pic.calc_Neval(sampler)
    print('Neval_cobaya = {0:d}'.format(Neval_cobaya))
    mnt.time_this(start_time)
else:
    print('Chains (hopefully) exist!')

In [ ]:
if Run_NN:
    start_time = time()
    updated_info_nn, sampler_nn = run(info_nn)
    Neval_nn = pic.calc_Neval(sampler_nn)
    print('Neval_nn = {0:d}'.format(Neval_nn))
    mnt.time_this(start_time)
else:
    print('Chains (hopefully) exist!')

In [ ]:
if Run_GPR:
    start_time = time()
    updated_info_gpr, sampler_gpr = run(info_gpr)
    Neval_gpr = pic.calc_Neval(sampler_gpr)
    print('Neval_gpr = {0:d}'.format(Neval_gpr))
    mnt.time_this(start_time)
else:
    print('Chains (hopefully) exist!')

In [ ]:
Save_Fig = True

In [ ]:
Burn_Frac = 0.3
rng = np.random.RandomState(42)
dim = n_params

start_time = time()

gd_sample = loadMCSamples(os.path.abspath(info["output"]),settings={'ignore_rows':Burn_Frac})
gd_sample.label = 'MCMC: {0:d} evals'.format(Neval_cobaya) if Run_Cobaya else 'MCMC'
# samples contain params | chi2 | chi2__name | ?? | ??
mcmc_covmat = gd_sample.getCovMat().matrix[:dim, :dim]
sample = gd_sample.samples
sample = sample.T
ibest = sample[-2].argmin()
mcmc_best = sample[:dim,ibest]
mcmc_chi2 = sample[-2,ibest]
pval = sysp.gammainc(mcmc_chi2/2,dof/2)
mcmc_sig = np.sqrt(np.diag(mcmc_covmat))
print('MCMC...')
print("... best fit ( a0,...a{0:d}) = ( ".format(n_params)+','.join(['%.4e' % (pval,) for pval in mcmc_best])+" )")
print("... std dev  ( a0,...a{0:d}) = ( ".format(n_params)+','.join(['%.4e' % (pval,) for pval in mcmc_sig])+" )")
print("... chi2_best,dof,chi2_red,pval: {0:.3f},{1:d},{2:.3f},{3:.3e}".format(mcmc_chi2,dof,mcmc_chi2/dof,pval))

gd_sample_nn = loadMCSamples(os.path.abspath(info_nn["output"]),settings={'ignore_rows':Burn_Frac})
gd_sample_nn.label = '     NN: {0:d} evals'.format(Xp.shape[1])
# samples contain params | chi2 | chi2__name | ?? | ??
mcmc_covmat_nn = gd_sample_nn.getCovMat().matrix[:dim, :dim]
sample_nn = gd_sample_nn.samples
sample_nn = sample_nn.T
ibest_nn = sample_nn[-2].argmin()
mcmc_best_nn = sample_nn[:dim,ibest_nn]
mcmc_chi2_nn = sample_nn[-2,ibest_nn]
pval_nn = sysp.gammainc(mcmc_chi2_nn/2,dof/2)
mcmc_sig_nn = np.sqrt(np.diag(mcmc_covmat_nn))
print('NN...')
print("... best fit ( a0,...a{0:d}) = ( ".format(n_params)+','.join(['%.4e' % (pval,) for pval in mcmc_best_nn])+" )")
print("... std dev  ( a0,...a{0:d}) = ( ".format(n_params)+','.join(['%.4e' % (pval,) for pval in mcmc_sig_nn])+" )")
print("... chi2_best,dof,chi2_red,pval: {0:.3f},{1:d},{2:.3f},{3:.3e}".format(mcmc_chi2_nn,dof,mcmc_chi2_nn/dof,pval_nn))

gd_sample_gpr = loadMCSamples(os.path.abspath(info_gpr["output"]),settings={'ignore_rows':Burn_Frac})
gd_sample_gpr.label = '   GPR: {0:d} evals'.format(gprt.pred_var.size)
# samples contain params | chi2 | chi2__name | ?? | ??
mcmc_covmat_gpr = gd_sample_gpr.getCovMat().matrix[:dim, :dim]
sample_gpr = gd_sample_gpr.samples
sample_gpr = sample_gpr.T
ibest_gpr = sample_gpr[-2].argmin()
mcmc_best_gpr = sample_gpr[:dim,ibest_gpr]
mcmc_chi2_gpr = sample_gpr[-2,ibest_gpr]
pval_gpr = sysp.gammainc(mcmc_chi2_gpr/2,dof/2)
mcmc_sig_gpr = np.sqrt(np.diag(mcmc_covmat_gpr))
print('GPR...')
print("... best fit ( a0,...a{0:d}) = ( ".format(n_params)+','.join(['%.4e' % (pval,) for pval in mcmc_best_gpr])+" )")
print("... std dev  ( a0,...a{0:d}) = ( ".format(n_params)+','.join(['%.4e' % (pval,) for pval in mcmc_sig_gpr])+" )")
print("... chi2_best,dof,chi2_red,pval: {0:.3f},{1:d},{2:.3f},{3:.3e}".format(mcmc_chi2_gpr,dof,mcmc_chi2_gpr/dof,pval_gpr))

plot_param_list = Params_List
Subplot_Size = 1.6 

gdplot = gdplt.get_subplot_plotter(subplot_size=Subplot_Size)
gdplot.settings.num_plot_contours = 3
gdplot.settings.axes_fontsize = FS3
gdplot.settings.axes_labelsize = FS2
gdplot.settings.title_limit_fontsize = FS3

gdplot.triangle_plot([gd_sample,gd_sample_nn,gd_sample_gpr], plot_param_list,
                     filled=[True,True,True],contour_colors=['indigo','crimson','darkgreen'],legend_loc='upper right',
                     title_limit=0)
for par_y in range(dim):
    str_y = plot_param_list[par_y]
    ax = gdplot.subplots[par_y,par_y]
    ax.axvline(mcmc_best[par_y],c='indigo',ls='--',lw=1,alpha=0.6)
    ax.axvline(mcmc_best_nn[par_y],c='crimson',ls='--',lw=1,alpha=0.6)
    ax.axvline(mcmc_best_gpr[par_y],c='darkgreen',ls='--',lw=1,alpha=0.6)
    for par_x in range(par_y):
        str_x = plot_param_list[par_x]
        #print(par_x,par_y,':',str_x,str_y)
        ax = gdplot.subplots[par_y,par_x]
        ax.scatter([mcmc_best[par_x]],[mcmc_best[par_y]],marker='*',s=50,c='aliceblue')
        ax.scatter([mcmc_best_nn[par_x]],[mcmc_best_nn[par_y]],marker='*',s=50,c='peachpuff')
        ax.scatter([mcmc_best_gpr[par_x]],[mcmc_best_gpr[par_y]],marker='*',s=50,c='lightcyan')
        ax.axvline(mcmc_best[par_x],c='indigo',ls='--',lw=1,alpha=0.6)
        ax.axhline(mcmc_best[par_y],c='indigo',ls='--',lw=1.5,alpha=0.6)
        ax.axvline(mcmc_best_nn[par_x],c='crimson',ls='--',lw=1,alpha=0.6)
        ax.axhline(mcmc_best_nn[par_y],c='crimson',ls='--',lw=1,alpha=0.6)
        ax.axvline(mcmc_best_gpr[par_x],c='darkgreen',ls='--',lw=1,alpha=0.6)
        ax.axhline(mcmc_best_gpr[par_y],c='darkgreen',ls='--',lw=1,alpha=0.6)

if Save_Fig:
    filename = 'contours_'+id_str+'.png'
    print('Writing to file: '+Plots_Dir+filename)
    gdplot.export(fname=filename,adir=Plots_Dir)

mnt.time_this(start_time)

In [ ]:
# MCMC
start_time = time()
model.params = mnt.cv(mcmc_best[:dim])
model_best = model.forward(mnt.X)[0]

N_Boot_Cobaya = np.min([1000,int(0.2*sample[0].size)])
Ind = gd_sample.random_single_samples_indices(random_state=42,max_samples=N_Boot_Cobaya)
N_Boot_Cobaya = Ind.size
print('N_Boot_Cobaya: ',N_Boot_Cobaya)

model_boot = np.zeros((N_Boot_Cobaya,n_samp),dtype=float)

print('... extracting stats from subsample')
for b in range(N_Boot_Cobaya):
    params_b = sample[:dim,Ind[b]] 
    model.params = mnt.cv(params_b[:dim])
    model_boot[b] = model.forward(mnt.X)[0]
    mnt.status_bar(b,N_Boot_Cobaya)

model_16pc = np.percentile(model_boot,16,axis=0)
model_84pc = np.percentile(model_boot,84,axis=0)

# NN
model.params = mnt.cv(mcmc_best_nn[:dim])
model_best_nn = model.forward(mnt.X)[0]

N_Boot_NN = np.min([1000,int(0.2*sample_nn[0].size)])
Ind_nn = gd_sample_nn.random_single_samples_indices(random_state=42,max_samples=N_Boot_NN)
N_Boot_NN = Ind_nn.size
print('N_Boot_NN: ',N_Boot_NN)

model_boot_nn = np.zeros((N_Boot_NN,n_samp),dtype=float)

print('... extracting stats from subsample')
for b in range(N_Boot_NN):
    params_b = sample_nn[:dim,Ind_nn[b]] 
    model.params = mnt.cv(params_b[:dim])
    model_boot_nn[b] = model.forward(mnt.X)[0]
    mnt.status_bar(b,N_Boot_NN)

model_16pc_nn = np.percentile(model_boot_nn,16,axis=0)
model_84pc_nn = np.percentile(model_boot_nn,84,axis=0)

# GPR
model.params = mnt.cv(mcmc_best_gpr[:dim])
model_best_gpr = model.forward(mnt.X)[0]

N_Boot_GPR = np.min([1000,int(0.2*sample_gpr[0].size)])
Ind_gpr = gd_sample_gpr.random_single_samples_indices(random_state=42,max_samples=N_Boot_GPR)
N_Boot_GPR = Ind_gpr.size
print('N_Boot_GPR: ',N_Boot_GPR)

model_boot_gpr = np.zeros((N_Boot_GPR,n_samp),dtype=float)

print('... extracting stats from subsample')
for b in range(N_Boot_GPR):
    params_b = sample_gpr[:dim,Ind_gpr[b]] 
    model.params = mnt.cv(params_b[:dim])
    model_boot_gpr[b] = model.forward(mnt.X)[0]
    mnt.status_bar(b,N_Boot_GPR)

model_16pc_gpr = np.percentile(model_boot_gpr,16,axis=0)
model_84pc_gpr = np.percentile(model_boot_gpr,84,axis=0)

del model_boot,model_boot_nn,model_boot_gpr
gc.collect()

cols = ['indigo','crimson','darkgreen']
plt.figure(figsize=(7,7))
plt.xlabel('x')
plt.ylabel('y')
if Model_Type == 'ps':
    plt.ylim(-1,6)
else:
    if n_comp == 1:
        plt.ylim(-1.5,1)
    else:
        plt.ylim(-1.5,3)
plt.plot(mnt.X[0],model_best,'-',lw=1,c=cols[0],label='MCMC')
plt.fill_between(mnt.X[0],model_84pc,model_16pc,color=cols[0],alpha=0.15)
plt.plot(mnt.X[0],model_best_nn,'-',lw=1,c=cols[1],label='NN')
plt.fill_between(mnt.X[0],model_84pc_nn,model_16pc_nn,color=cols[1],alpha=0.15)
plt.plot(mnt.X[0],model_best_gpr,'-',lw=1,c=cols[2],label='GPR')
plt.fill_between(mnt.X[0],model_84pc_gpr,model_16pc_gpr,color=cols[2],alpha=0.15)

plt.errorbar(mnt.X[0],mnt.Y[0],yerr=sigma,c='k',ls='none',capsize=5,marker='o',markersize=4,label='data')

plt.legend(loc='upper left')
plt.minorticks_on()
if Save_Fig:
    filename = Plots_Dir+'stats_'+id_str+'.png'
    print('Writing to file: '+filename)
    plt.savefig(filename,bbox_inches='tight')
plt.show()

mnt.time_this(start_time)

In [ ]:
# Build = True

# Train_Frac = 0.8
# Val_Frac = 0.1
# Min_Layer = 2
# Max_Layer = 2
# Max_Ex = 2*n_params
# TTL = 1e-5
# Loss_Type = 'square'
# Seed = None
# FileStem = 'net/NN_'+id_str

# bnn = BuildNN(Xp,Yp,train_frac=Train_Frac,val_frac=Val_Frac,min_layer=Min_Layer,max_layer=Max_Layer,max_ex=Max_Ex,
#               target_test_loss=TTL,loss_type=Loss_Type,seed=Seed,file_stem=FileStem)
# if Build:
#     start_time = time()
#     net,params_train,mtl = bnn.trainNN()
#     print('Best mean test loss = {0:.3e}'.format(mtl))
#     print('Setup params: ',net.params)
#     print('Training params: ',params_train)
#     bnn.time_this(start_time)
# else:
#     net = bnn.load()
#     print('Loaded network with')
#     print('...    setup params: ',net.params)
#     params_train = bnn.load_train()
#     print('... training params: ',params_train)

In [ ]:
# if Build:
#     plt.figure(figsize=(7,7))
#     plt.xlim(0.5,params_train['max_epoch'])
#     plt.ylim(1e0,1e7)
#     plt.yscale('log')
#     plt.xscale('log')
#     nn_label = '{0:1d}-layer NN'.format(net.L)
#     # bn_label = '{0:1d}-layer NN with BatchNorm'.format(net.L)
#     plt.xlabel('epoch')
#     plt.ylabel('loss')
#     plt.plot(net.epochs,net.epoch_loss,'k-',lw=0.5,label=nn_label+': train')
#     plt.plot(net.epochs,net.val_loss,'r-',lw=0.5,label=nn_label+': val')
#     # plt.plot(net_bn.epochs,net_bn.epoch_loss,'r-',lw=0.5,label=bn_label)
#     plt.legend()
#     plt.show()
#     print(net.epoch_loss[-1])